In [ ]:
# Установки библиотек

In [5]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install ultralytics

Note: you may need to restart the kernel to use updated packages.


In [7]:
# import system libs
import os
import time
import shutil
import pathlib
import itertools

# import data handling tools
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style('darkgrid')
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# import Deep learning Libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras import regularizers
from ultralytics import YOLO
# Ignore Warnings
import warnings
warnings.filterwarnings("ignore")

print ('modules loaded')

modules loaded


In [13]:
pip install roboflow

  Using cached roboflow-1.1.65-py3-none-any.whl.metadata (9.7 kB)
  Using cached opencv_python_headless-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached roboflow-1.1.65-py3-none-any.whl (85 kB)
Using cached opencv_python_headless-4.10.0.84-cp37-abi3-win_amd64.whl (38.8 MB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] Отказано в доступе: 'D:\\anaconda 2\\Lib\\site-packages\\cv2\\cv2.pyd'
Consider using the `--user` option or check the permissions.



In [7]:
#разархивирование

In [8]:
import zipfile
zip_path = "archive (7).zip"  # или полный путь (/content/archive.zip в Colab)
extract_dir = "extracted_folder4"

In [ ]:
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)
print("Архив распакован!")

In [ ]:
# Создание датафреймов из изображений и меток и перенос изображений в файлы для модели yolo

In [15]:
def define_paths(data_dir):
    filepaths = []# Сюда будем складывать пути к файлам
    labels = []# Сюда соответствующие метки

    folds = os.listdir(data_dir)# Получаем список всего в папке
    for fold in folds:
        foldpath = os.path.join(data_dir, fold)
        
        if pathlib.Path(foldpath).suffix != '':# Если нет расширения(txt...), то пропускаем
            continue

        filelist = os.listdir(foldpath)
        for file in filelist:
            fpath = os.path.join(foldpath, file)


            if pathlib.Path(foldpath).suffix == '':# Если нет расширения, то пропускаем
                
                if pathlib.Path(fpath).parts[-1] == 'masks' or pathlib.Path(fpath).parts[-1] == 'Masks' or pathlib.Path(fpath).parts[-1] == 'MASKS': #Берем путь, разбиваем, и если последний элемент, маска, то пропускаем
                    continue

                else:
                    o_file = os.listdir(fpath)
                    for f in o_file:
                        ipath = os.path.join(fpath, f)
                        filepaths.append(ipath)
                        labels.append(fold)

            else:
                filepaths.append(fpath)
                labels.append(fold)

    return filepaths, labels

In [17]:
# Объединить пути данных с метками в один фрейм данных (для последующего встраивания в модель)
def define_df(files, classes):
    Fseries = pd.Series(files, name= 'filepaths')
    Lseries = pd.Series(classes, name='labels')
    return pd.concat([Fseries, Lseries], axis= 1)
def split_data(data_dir):
    # train dataframe
    files, classes = define_paths(data_dir)
    df = define_df(files, classes)
    strat = df['labels']
    train_df, dummy_df = train_test_split(df,  train_size= 0.8, shuffle= True, random_state= 123, stratify= strat)

    # valid and test dataframe
    strat = dummy_df['labels']
    valid_df, test_df = train_test_split(dummy_df,  train_size= 0.5, shuffle= True, random_state= 123, stratify= strat)

    return train_df, valid_df, test_df


In [19]:
train_df, valid_df, test_df=split_data("extracted_folder4/COVID-19_Radiography_Dataset")

In [20]:
import os
import cv2
import pandas as pd
from ultralytics import YOLO
from tqdm import tqdm

def prepare_yolo_dataset(train_df, valid_df, test_df):
    # Создаем структуру папок для YOLO классификации
    os.makedirs('yolo_dataset6/train/COVID', exist_ok=True)
    os.makedirs('yolo_dataset6/train/Normal', exist_ok=True)
    os.makedirs('yolo_dataset6/train/Lung_Opacity', exist_ok=True)
    os.makedirs('yolo_dataset6/train/Viral Pneumonia', exist_ok=True)

    os.makedirs('yolo_dataset6/val/COVID', exist_ok=True)
    os.makedirs('yolo_dataset6/val/Normal', exist_ok=True)
    os.makedirs('yolo_dataset6/val/Lung_Opacity', exist_ok=True)
    os.makedirs('yolo_dataset6/val/Viral Pneumonia', exist_ok=True)

    os.makedirs('yolo_dataset6/test/COVID', exist_ok=True)
    os.makedirs('yolo_dataset6/test/Normal', exist_ok=True)
    os.makedirs('yolo_dataset6/test/Lung_Opacity', exist_ok=True)
    os.makedirs('yolo_dataset6/test/Viral Pneumonia', exist_ok=True)

    # Функция для копирования файлов из датафрейма в целевую директорию
    def copy_files(df, dataset_type):
        for idx, row in tqdm(df.iterrows(), total=len(df), desc=f"Processing {dataset_type} set"):
            src_path = row['filepaths']
            label = row['labels']

            # Читаем изображение
            img = cv2.imread(src_path)
            if img is None:
                print(f"Warning: Could not read image {src_path}")
                continue

            # Создаем имя файла без пути
            img_name = os.path.basename(src_path)

            # Сохраняем в соответствующую директорию
            dest_path = f'yolo_dataset6/{dataset_type}/{label}/{img_name}'
            cv2.imwrite(dest_path, img)

    # Обрабатываем все датафреймы
    copy_files(train_df, 'train')
    copy_files(valid_df, 'val')
    copy_files(test_df, 'test')



In [23]:
#так как классы распределены не поровну, берем все изображений из самого маленького класса и по опрделенному количеству из остальных классов

In [25]:
import pandas as pd
import numpy as np

def create_balanced_subset(train_df, target_size=12000):
    # Находим миноритарный класс
    class_counts = train_df['labels'].value_counts()
    minority_class = class_counts.idxmin()
    minority_count = class_counts.min()
    
    print(f"Миноритарный класс: '{minority_class}' ({minority_count} примеров)")
    
    # Берём ВСЕ примеры миноритарного класса
    minority_df = train_df[train_df['labels'] == minority_class]
    
    # Вычисляем сколько взять из других классов
    remaining_size = target_size - len(minority_df)
    other_classes = [cls for cls in class_counts.index if cls != minority_class]
    samples_per_class = int(remaining_size / len(other_classes))
    
    print(f"Берём по {samples_per_class} примеров из классов: {other_classes}")
    
    # Собираем датасет
    dfs = [minority_df]
    for cls in other_classes:
        cls_df = train_df[train_df['labels'] == cls]
        dfs.append(cls_df.sample(n=min(samples_per_class, len(cls_df)), 
                         random_state=42))
    
    balanced_df = pd.concat(dfs).sample(frac=1, random_state=42)  # Перемешиваем
    print(f"Итоговый размер датасета: {len(balanced_df)}")
    return balanced_df

# Пример использования
balanced_train_df = create_balanced_subset(train_df, target_size=12000)
prepare_yolo_dataset(balanced_train_df, valid_df, test_df)

Миноритарный класс: 'Viral Pneumonia' (1076 примеров)
Берём по 3641 примеров из классов: ['Normal', 'Lung_Opacity', 'COVID']
Итоговый размер датасета: 11251


Processing test set: 100%|████████████████████████████████████████████████████████| 2117/2117 [00:13<00:00, 158.20it/s]


In [26]:
#Создание модели

In [27]:
from tqdm import tqdm
def train_yolo_model():
    # Загружаем модель с предобученными весами
    model = YOLO('yolov8x-cls.pt')
    
    # Словарь для хранения метрик
    metrics = {
        'epoch': [],
        'train_acc': [],
        'val_acc': [],
        'train_loss': [],
        'val_loss': []
    }

    # Прогресс-бар с динамическим обновлением
    pbar = tqdm(total=14, desc='Training Progress', leave=True)

    # Callback для сохранения метрик
    def on_train_epoch_end(trainer):
        current_epoch = trainer.epoch
        metrics['epoch'].append(current_epoch)
        metrics['train_acc'].append(trainer.metrics.get('metrics/accuracy_top1', 0))
        metrics['train_loss'].append(trainer.metrics.get('train/loss', 0))
        metrics['val_acc'].append(trainer.metrics.get('val/accuracy_top1', 0))
        metrics['val_loss'].append(trainer.metrics.get('val/loss', 0))
        
        # Обновляем прогресс-бар
        pbar.set_postfix({
            'trn_acc': f"{metrics['train_acc'][-1]:.3f}",
            'val_acc': f"{metrics['val_acc'][-1]:.3f}",
            'trn_loss': f"{metrics['train_loss'][-1]:.3f}"
        })
        pbar.update(1)

    model.add_callback("on_train_epoch_end", on_train_epoch_end)

    # Оптимизированные параметры обучения
    try:
        results = model.train(
            data='yolo_dataset6',
            imgsz=256,
            epochs=10,
            batch=64,
            optimizer='AdamW',
            lr0=0.001,  # Уменьшенный learning rate
            lrf=0.02,    # Final LR = lr0 * lrf
            warmup_epochs=2,  # Плавный старт
            weight_decay=0.0012,  # Увеличенная регуляризация
            dropout=0.23,  # Больший dropout
            label_smoothing=0.08,  # Сглаживание меток
            patience=15,   # Увеличенное терпение для early stoppin
            degrees=0,
            scale=0.04, 
            translate=0.07,
            nbs=64,
            val=True,
            device='cuda' if torch.cuda.is_available() else 'cpu',
            name='stable_covid_model',
            amp=True      # Mixed precision
        )
    except Exception as e:
        print(f"Training error: {str(e)}")
        return None, None

    pbar.close()

    # Визуализация результатов
    if metrics['epoch']:
        plt.figure(figsize=(16, 6))
        
        # График точности
        plt.subplot(1, 2, 1)
        plt.plot(metrics['epoch'], metrics['train_acc'], 'b-', label='Train Accuracy')
        plt.plot(metrics['epoch'], metrics['val_acc'], 'g-', label='Validation Accuracy')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.title('Training and Validation Accuracy')
        plt.legend()
        plt.grid(True)
        
        # График потерь
        plt.subplot(1, 2, 2)
        plt.plot(metrics['epoch'], metrics['train_loss'], 'r-', label='Train Loss')
        plt.plot(metrics['epoch'], metrics['val_loss'], 'm-', label='Validation Loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.title('Training and Validation Loss')
        plt.legend()
        plt.grid(True)
        
        plt.tight_layout()
        plt.savefig('training_metrics.png', dpi=300)
        plt.show()

    return model, metrics
import torch
model, metrics = train_yolo_model()

# Сохранение модели
if model is not None:
    model.save('best_covid_model.pt')
    print("Модель успешно сохранена как 'best_covid_model.pt'")



Training Progress:   0%|                                                                        | 0/14 [00:00<?, ?it/s]

New https://pypi.org/project/ultralytics/8.3.146 available  Update with 'pip install -U ultralytics'
WARNING 'label_smoothing' is deprecated and will be removed in in the future.
Ultralytics 8.3.137  Python-3.12.7 torch-2.7.0+cpu CPU (11th Gen Intel Core(TM) i5-11260H 2.60GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=yolo_dataset6, degrees=0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.23, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=256, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.02, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8x-cls.pt, momentum=0.937, mosaic

Training Progress:  64%|███████▋    | 9/14 [9:10:03<5:05:35, 3667.03s/it, trn_acc=0.955, val_acc=0.000, trn_loss=0.000]

val: C:\Users\User\yolo_dataset6\val... found 2116 images in 4 classes  
test: C:\Users\User\yolo_dataset6\test... found 2117 images in 4 classes  
Overriding model.yaml nc=1000 with nc=4

                   from  n    params  module                                       arguments                     


  0                  -1  1      2320  ultralytics.nn.modules.conv.Conv             [3, 80, 3, 2]                 
  1                  -1  1    115520  ultralytics.nn.modules.conv.Conv             [80, 160, 3, 2]               
  2                  -1  3    436800  ultralytics.nn.modules.block.C2f             [160, 160, 3, True]           
  3                  -1  1    461440  ultralytics.nn.modules.conv.Conv             [160, 320, 3, 2]              
  4                  -1  6   3281920  ultralytics.nn.modules.block.C2f             [320, 320, 6, True]           
  5                  -1  1   1844480  ultralytics.nn.modules.conv.Conv             [320, 640, 3, 2]              
  6                  -1  6  13117440  ultralytics.nn.modules.block.C2f             [640, 640, 6, True]           
  7                  -1  1   7375360  ultralytics.nn.modules.conv.Conv             [640, 1280, 3, 2]             
  8                  -1  3  27865600  ultralytics.nn.modules.block.C2f             [1280

...


               classes   top1_acc   top5_acc: 100%|██████████| 17/17 [03:03<00:00, 10.77s/it]


                   all      0.963          1
Speed: 0.0ms preprocess, 83.0ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs\classify\stable_covid_model35


Training Progress:  71%|███████▊   | 10/14 [9:25:09<3:46:03, 3390.93s/it, trn_acc=0.963, val_acc=0.000, trn_loss=0.000]


<Figure size 1600x600 with 2 Axes>

Модель успешно сохранена как 'best_covid_model.pt'


In [8]:
model = YOLO("best_covid_model.pt")

In [10]:
results = model.val(data='yolo_dataset6/test')

Ultralytics 8.3.137  Python-3.12.7 torch-2.7.0+cpu CPU (11th Gen Intel Core(TM) i5-11260H 2.60GHz)
YOLOv8x-cls summary (fused): 54 layers, 56,128,324 parameters, 0 gradients, 153.8 GFLOPs
WARNING Dataset 'split=train' not found at C:\Users\User\yolo_dataset6\test\train
Found 2117 images in subdirectories. Attempting to split...
Splitting C:\Users\User\yolo_dataset6\test (4 classes, 2117 images) into 80% train, 20% val...
Split complete in C:\Users\User\yolo_dataset6\test_split 
train: C:\Users\User\yolo_dataset6\test_split\train... found 2117 images in 4 classes  
val: C:\Users\User\yolo_dataset6\test_split\val... found 1436 images in 4 classes  
test: None...
val: Fast image access  (ping: 0.10.1 ms, read: 138.577.7 MB/s, size: 101.8 KB)


val: Scanning C:\Users\User\yolo_dataset6\test_split\val... 1436 images, 0 corrupt: 100%|██████████| 1436/1436 [00:02<0

val: New cache created: C:\Users\User\yolo_dataset6\test_split\val.cache



               classes   top1_acc   top5_acc: 100%|██████████| 90/90 [01:41<00:00,  1.12s/it]


                   all      0.971          1
Speed: 0.0ms preprocess, 67.0ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs\classify\val2
